In [1]:
import pandas as pd
import psycopg2
import warnings

DBNAME = 
USER = 
PASSWORD = 
HOST = 
PORT = 

connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

warnings.filterwarnings('ignore')

In [2]:
query_5_1 = f'''
        WITH cte AS
        (SELECT e.name as employee_name, 
        row_number() over (order by count(e.name) DESC) AS rn 
        
        FROM vacancies AS v 
        JOIN employers AS e ON v.employer_id = e.id 
        
        GROUP BY 
        e.name) 
        
        SELECT c.employee_name 
        
        FROM cte AS c 
        
        WHERE rn IN (1, 5)
'''

df_1 = pd.read_sql_query(query_5_1, connection)
display(df_1)

,employee_name
0,Яндекс
1,Газпром нефть


In [3]:
tables = pd.read_html('https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8')
cities_df = tables[1]
cities_list = tuple(cities_df['Город'])

query_5_2 = f'''
        WITH cte AS 
        (SELECT a.name, 
        COUNT(v.id) AS cnt
        
        FROM vacancies AS v 
        JOIN employers AS e ON e.id = v.employer_id AND e.name = 'Яндекс' 
        JOIN areas AS a on a.id = v.area_id AND a.name IN {cities_list} 
        
        GROUP BY 
        a.name) 
        
        SELECT 
        'Total' AS name, 
        SUM(cnt) 
        
        FROM cte AS c 
        UNION ALL 
        
        SELECT c.name, 
        c.cnt 
        
        FROM cte AS c
'''

df_2 = pd.read_sql_query(query_5_2, connection)
display(df_2)

,name,sum
0,Total,485.0
1,Волгоград,24.0
2,Воронеж,32.0
3,Екатеринбург,39.0
4,Казань,25.0
5,Краснодар,30.0
6,Красноярск,23.0
7,Москва,54.0
8,Нижний Новгород,36.0
9,Новосибирск,35.0
